# QAOA Warm-Start GPU: Portfolio Optimization with Heuristic Initialization

**Objetivo**: Implementar QAOA para optimización de portafolio usando inicializaciones warm-start desde **heurísticas financieras clásicas**, no desde la solución óptima.

## ¿Qué son los Warm-Starts Heurísticos?

En lugar de inicializar el estado cuántico aleatoriamente, partimos desde **soluciones razonables pero sub-óptimas** obtenidas con métodos greedy clásicos:

1. **Greedy-Return**: Seleccionar los B activos con mayor retorno esperado individual
2. **Greedy-Risk**: Seleccionar los B activos con menor volatilidad individual  
3. **Greedy-Sharpe**: Seleccionar los B activos con mejor ratio Sharpe individual

**¿Por qué estas heurísticas NO son óptimas?**
- Ignoran las **correlaciones** entre activos (diversificación)
- No consideran la **matriz de covarianza** completa
- Son aproximaciones miopes (greedy) sin visión global

**Objetivo de QAOA**: Partir de estas soluciones sub-óptimas (~60-80% de gap) y explorar el espacio cuántico para **encontrar soluciones mejores o cercanas al óptimo global**.

---

## Características del Notebook

✅ **3 warm-starts heurísticos** (sin usar brute-force - eso sería trampa)  
✅ **XY mixer** para preservar cardinalidad  
✅ **GPU acceleration** (si está disponible)  
✅ **Benchmark honesto** vs brute-force para medir mejora real

---


In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.circuit.library import RXXGate, RYYGate, RZZGate
from qiskit_aer import AerSimulator

from scipy.optimize import minimize

import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All libraries imported")


✓ All libraries imported


## 1. Configuration


In [25]:
CONFIG = {
    'p_layers': 6,
    'shots': 8000,
    'seed': 7,
    'n_warm_starts': 3,
    'optimizer': 'COBYLA',
    'max_iter': 250,
    'rhobeg': 0.5,
    'gpu_enabled': True
}

P_LAYERS = CONFIG['p_layers']
SHOTS = CONFIG['shots']
SEED = CONFIG['seed']

rng = np.random.default_rng(SEED)

print("="*55)
print("       QAOA - SOLO WARM-STARTS (NO RANDOM-STARTS)")
print("="*55)
for key, val in CONFIG.items():
    print(f"  {key:21s}: {val}")
print("="*55)


       QAOA - SOLO WARM-STARTS (NO RANDOM-STARTS)
  p_layers             : 6
  shots                : 8000
  seed                 : 7
  n_warm_starts        : 3
  optimizer            : COBYLA
  max_iter             : 250
  rhobeg               : 0.5
  gpu_enabled          : True


In [26]:
data = np.load("portfolio_qubo_data.npz", allow_pickle=True)
Q = data['Q']
q = data['q']
mu = data['mu']
Sigma = data['Sigma']
B = int(data['B'])
TICKERS = list(data['TICKERS'])
n = len(TICKERS)
print(f"✓ Loaded QUBO data: {n} assets, {B} to select")



✓ Loaded QUBO data: 21 assets, 4 to select


In [27]:
def qubo_to_ising(Q, q):
    n = len(q)
    J = np.zeros((n, n))
    h = np.zeros(n)
    const = 0.0
    for i in range(n):
        for j in range(i+1, n):
            Jij = Q[i, j] / 4.0
            J[i, j] = Jij
            h[i] += -Q[i, j] / 4.0
            h[j] += -Q[i, j] / 4.0
            const += Q[i, j] / 4.0
    for i in range(n):
        h[i] += -(Q[i, i] / 2.0) - (q[i] / 2.0)
        const += (Q[i, i] / 2.0) + (q[i] / 2.0)
    return J, h, const

J, h, const_shift = qubo_to_ising(Q, q)
print(f"✓ QUBO→Ising transformation complete")


✓ QUBO→Ising transformation complete


In [28]:
def f_qubo(x):
    return float(x @ Q @ x + q @ x)

def bitarray_from_qiskit_string(s):
    return np.array([int(c) for c in s[::-1]], dtype=int)

def bind_params(circ, mapping):
    return circ.assign_parameters(mapping, inplace=False)

def random_theta():
    return rng.uniform(0, 2*np.pi, size=2*P_LAYERS)

def is_valid(x, B):
    return np.sum(x) == B


## Warm-Start Initializations: Heurísticas Financieras

Generamos 3 carteras iniciales usando estrategias greedy simples que **NO requieren optimización cuadrática**:

### 1. Greedy-Return (Máximo Retorno)
- Criterio: Seleccionar top B activos con mayor $\mu_i$
- Lógica: "Invertir en los que más ganan"
- Problema: Ignora riesgo y correlaciones
- Gap esperado vs óptimo: ~60-80%

### 2. Greedy-Risk (Mínimo Riesgo)
- Criterio: Seleccionar top B activos con menor $\sigma_i^2$
- Lógica: "Invertir en los más estables"
- Problema: Ignora retorno y diversificación
- Gap esperado vs óptimo: ~70-90%

### 3. Greedy-Sharpe (Mejor Ratio Individual)
- Criterio: Seleccionar top B activos con mayor $\mu_i / \sigma_i$
- Lógica: "Invertir en los de mejor retorno ajustado por riesgo"
- Problema: Sharpe individual ≠ Sharpe de portafolio
- Gap esperado vs óptimo: ~50-70%

**Nota importante**: Ninguna de estas heurísticas considera la matriz de covarianza completa $\Sigma$, por lo que **garantizadamente son sub-óptimas** para el problema QUBO real.

QAOA intentará **mejorar estas soluciones** explorando el espacio de Hilbert.


In [29]:
initial_states = []
labels = []

# Warm-start 1: greedy máximo retorno esperado
returns_sorted = np.argsort(-mu)
greedy_return = np.zeros(n, dtype=int)
greedy_return[returns_sorted[:B]] = 1
initial_states.append(greedy_return)
labels.append("Greedy-Return")
print("✓ Warm-start 1: greedy max return")
print("  Assets:", [TICKERS[i] for i in returns_sorted[:B]])
print("  Cost:", f_qubo(greedy_return))

# Warm-start 2: greedy mínimo riesgo individual
risks_sorted = np.argsort(np.diag(Sigma))
greedy_risk = np.zeros(n, dtype=int)
greedy_risk[risks_sorted[:B]] = 1
initial_states.append(greedy_risk)
labels.append("Greedy-Risk")
print("\n✓ Warm-start 2: greedy min risk")
print("  Assets:", [TICKERS[i] for i in risks_sorted[:B]])
print("  Cost:", f_qubo(greedy_risk))

# Warm-start 3: greedy máximo Sharpe ratio individual
# Sharpe individual aproximado: mu_i / sqrt(Sigma_ii)
sharpe_individual = mu / np.sqrt(np.diag(Sigma) + 1e-10)
sharpe_sorted = np.argsort(-sharpe_individual)
greedy_sharpe = np.zeros(n, dtype=int)
greedy_sharpe[sharpe_sorted[:B]] = 1
initial_states.append(greedy_sharpe)
labels.append("Greedy-Sharpe")
print("\n✓ Warm-start 3: greedy max Sharpe")
print("  Assets:", [TICKERS[i] for i in sharpe_sorted[:B]])
print("  Cost:", f_qubo(greedy_sharpe))

print(f"\nTotal initializations: {len(initial_states)} (all heuristic warm-starts)")



✓ Warm-start 1: greedy max return
  Assets: ['WMT', 'JNJ', 'AAPL', 'NVDA']
  Cost: 0.8145010302137972

✓ Warm-start 2: greedy min risk
  Assets: ['AVGO', 'CAT', 'UNH', 'TSLA']
  Cost: 0.16148146981731318

✓ Warm-start 3: greedy max Sharpe
  Assets: ['JNJ', 'AAPL', 'WMT', 'NVDA']
  Cost: 0.8145010302137972

Total initializations: 3 (all heuristic warm-starts)


In [30]:
def build_qaoa_xy(n, P, J, h, init_bits):
    qc = QuantumCircuit(n, name="QAOA_XY")
    for i, bit in enumerate(init_bits):
        if bit == 1:
            qc.x(i)
    gammas = [Parameter(f"γ_{k}") for k in range(P)]
    betas = [Parameter(f"β_{k}") for k in range(P)]
    ring_pairs = [(i, (i+1) % n) for i in range(n)]
    for layer in range(P):
        γ = gammas[layer]
        β = betas[layer]
        for i in range(n):
            if abs(h[i]) > 1e-15:
                qc.rz(2.0 * γ * h[i], i)
        for i in range(n):
            for j in range(i+1, n):
                if abs(J[i, j]) > 1e-15:
                    qc.append(RZZGate(2.0 * γ * J[i, j]), [i, j])
        for (i, j) in ring_pairs:
            qc.append(RXXGate(2.0 * β), [i, j])
            qc.append(RYYGate(2.0 * β), [i, j])
    return qc, gammas + betas


In [31]:
print("Setting up backend...")
try:
    if CONFIG['gpu_enabled']:
        try:
            backend = AerSimulator(device='GPU', method='statevector')
            test_qc = QuantumCircuit(2)
            test_qc.h(0)
            test_qc.measure_all()
            backend.run(test_qc, shots=10).result()
            print("✓ GPU backend initialized")
        except:
            backend = AerSimulator(method='statevector')
            print("✓ CPU backend initialized (no GPU available)")
    else:
        backend = AerSimulator(method='statevector')
        print("✓ CPU backend initialized")
except:
    backend = AerSimulator(method='statevector')
    print("✓ CPU backend initialized")


Setting up backend...
✓ CPU backend initialized (no GPU available)


In [32]:
def get_objective(ansatz, theta_params, t_ansatz):
    qaoa_trace = []
    iteration_count = [0]
    def objective(theta):
        param_dict = {p: float(t) for p, t in zip(theta_params, theta)}
        circ = bind_params(t_ansatz, param_dict)
        result = backend.run(circ, shots=SHOTS, seed_simulator=SEED).result()
        counts = result.get_counts()
        total_cost = 0.0
        valid = 0
        for bitstring, count in counts.items():
            x = bitarray_from_qiskit_string(bitstring)
            if is_valid(x, B):
                total_cost += count * f_qubo(x)
                valid += count
        if valid == 0:
            return qaoa_trace[-1] if qaoa_trace else 0.0
        avg_cost = total_cost / valid
        qaoa_trace.append(avg_cost)
        iteration_count[0] += 1
        if iteration_count[0] % 50 == 0:
            print(f"    Iter {iteration_count[0]:4d}: cost = {avg_cost:.6f}, valid = {valid}/{SHOTS}")
        return avg_cost
    return objective


In [33]:
print("\n" + "="*60)
print("          QAOA: SOLO WARM-STARTS")
print("="*60)
results = []
best_val = np.inf
best_theta = None
best_idx = None
t_total_start = time.perf_counter()

for i, (init_bits, label) in enumerate(zip(initial_states, labels)):
    print(f"\n[{i+1}/3] WARM-START ({label})")
    ansatz, theta_params = build_qaoa_xy(n, P_LAYERS, J, h, init_bits)
    ansatz_meas = ansatz.copy()
    ansatz_meas.measure_all()
    t_ansatz = transpile(ansatz_meas, backend, optimization_level=1, seed_transpiler=SEED)
    objective = get_objective(ansatz, theta_params, t_ansatz)
    x0 = rng.uniform(0, 2*np.pi, size=2*P_LAYERS)
    opt_result = minimize(
        objective, x0, method='COBYLA',
        options={'maxiter': CONFIG['max_iter'], 'rhobeg': CONFIG['rhobeg']}
    )
    print(f"  Final cost: {opt_result.fun:.6f} ({opt_result.nfev} evals)")
    results.append({
        'label': label,
        'final_cost': opt_result.fun,
        'final_theta': opt_result.x,
        'nfev': opt_result.nfev,
        'ansatz': ansatz,
        'theta_params': theta_params,
        'init_bits': init_bits.copy()
    })
    if opt_result.fun < best_val:
        best_val = opt_result.fun
        best_theta = opt_result.x
        best_idx = i

t_total_end = time.perf_counter()
print(f"\nAll optimizations complete in {t_total_end-t_total_start:.2f}s")
print(f"Best found at idx={best_idx+1} ({results[best_idx]['label']}) with cost {best_val:.6f}")

best_ansatz = results[best_idx]['ansatz']
best_theta_params = results[best_idx]['theta_params']



          QAOA: SOLO WARM-STARTS

[1/3] WARM-START (Greedy-Return)
    Iter   50: cost = 0.273261, valid = 8000/8000
    Iter  100: cost = 0.267122, valid = 8000/8000
  Final cost: 0.266425 (145 evals)

[2/3] WARM-START (Greedy-Risk)
    Iter   50: cost = 0.325592, valid = 8000/8000
    Iter  100: cost = 0.323161, valid = 8000/8000
  Final cost: 0.323158 (130 evals)

[3/3] WARM-START (Greedy-Sharpe)
    Iter   50: cost = 0.305164, valid = 8000/8000
    Iter  100: cost = 0.299551, valid = 8000/8000
  Final cost: 0.299419 (144 evals)

All optimizations complete in 353.13s
Best found at idx=1 (Greedy-Return) with cost 0.266425


In [34]:
print("\nSampling with best found parameters...")

best_init_bits = results[best_idx]['init_bits']
best_circuit, best_theta_params = build_qaoa_xy(n, P_LAYERS, J, h, best_init_bits)
circuit_measured = best_circuit.copy()
circuit_measured.measure_all()
t_trans = transpile(circuit_measured, backend, optimization_level=1, seed_transpiler=SEED)
final_params = {p: float(t) for p, t in zip(best_theta_params, best_theta)}
final_circ = bind_params(t_trans, final_params)

res = backend.run(final_circ, shots=SHOTS, seed_simulator=SEED).result()
counts = res.get_counts()

valid_solutions = []
for bitstring, count in counts.items():
    x = bitarray_from_qiskit_string(bitstring)
    if is_valid(x, B):
        cost = f_qubo(x)
        valid_solutions.append((bitstring, count, cost, x))

valid_solutions.sort(key=lambda t: t[2])
s_best, c_best, fx_best, x_best = valid_solutions[0]
sel_idx = np.where(x_best == 1)[0]
sel_tickers = [TICKERS[i] for i in sel_idx]

w = np.zeros(n)
w[sel_idx] = 1.0 / B
mu_ann = 252 * float(mu@w)
std_ann = np.sqrt(252 * float(w@Sigma@w))
sharpe = mu_ann / std_ann if std_ann > 1e-6 else 0.0

print(f"\n{'='*60}")
print("        FINAL QAOA WARM-START RESULTS")
print(f"{'='*60}")
print(f"Bitstring: {s_best}  QUBO cost: {fx_best:.6f}")
print("Portfolio:", ", ".join(sel_tickers))
print(f"Annualized Return: {mu_ann*100:.2f}%   Volatility: {std_ann*100:.2f}%   Sharpe: {sharpe:.3f}")



Sampling with best found parameters...

        FINAL QAOA WARM-START RESULTS
Bitstring: 011000000000010100000  QUBO cost: 0.147206
Portfolio: AVGO, TSLA, CAT, VZ
Annualized Return: 2.80%   Volatility: 17.03%   Sharpe: 0.165


In [35]:
try:
    bf = np.load("bruteforce_results.npz", allow_pickle=True)
    fx_bf = float(bf['fx_bruteforce'])
    gap = fx_best - fx_bf
    gap_r = 100*gap/fx_bf if fx_bf != 0 else np.nan
    print("\nComparison to brute-force:")
    print(f"  QAOA cost:  {fx_best:.6f}")
    print(f"  BF  cost:   {fx_bf:.6f}")
    print(f"  Absolute gap: {gap:.6f}")
    print(f"  Relative gap: {gap_r:.2f}%")
except Exception as e:
    print("Brute-force data not loaded:", e)



Comparison to brute-force:
  QAOA cost:  0.147206
  BF  cost:   0.147206
  Absolute gap: 0.000000
  Relative gap: 0.00%
